In [1]:
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import single,ward,complete,average,centroid,cophenet
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import tqdm

In [2]:
from COS_Funcs.utils import read_data
from COS_Funcs.baseline.baseline import datasets

C:\Program Files (x86)\ANACONDA\lib\site-packages\_ctgan\synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# Not Normed
linkages = ['single','complete','average','centroid','ward']
df = pd.DataFrame(columns= linkages + ['best','worst'],index=datasets)

for dataset in tqdm.tqdm(datasets):
    X,y = read_data(dataset,norm=True)
#     X = X[y==1] # only min data
    cond_m = pdist(X)
    
    # linkages
    single_ = single(cond_m)
    complete_ = complete(cond_m)
    average_ = average(cond_m)
    centroid_ = centroid(cond_m)
    ward_ = ward(cond_m)
    
    # cophenetic correlation coefficient
    single_cp = cophenet(single_,cond_m)[0]
    complete_cp = cophenet(complete_,cond_m)[0]
    average_cp = cophenet(average_,cond_m)[0]
    centroid_cp = cophenet(centroid_,cond_m)[0]
    ward_cp = cophenet(ward_,cond_m)[0]
    
    # calculate best and worst
    coeffs = [single_cp,complete_cp,average_cp,centroid_cp,ward_cp]
    index_best = coeffs.index(max(coeffs))
    index_worst = coeffs.index(min(coeffs))
    b_w = [linkages[index_best],linkages[index_worst]]
    
    df.loc[dataset] = [single_cp,complete_cp,average_cp,centroid_cp,ward_cp]+b_w
    
df

100%|██████████| 75/75 [00:54<00:00,  1.37it/s]


,single,complete,average,centroid,ward,best,worst
Datasets\abalone11–17.csv,0.301019,0.622621,0.687756,0.656912,0.579886,average,single
Datasets\abalone19.csv,0.420828,0.672375,0.713046,0.660262,0.605307,average,single
Datasets\abalone4–8.csv,0.525394,0.636629,0.759979,0.795159,0.585441,centroid,single
Datasets\abalone5–10.csv,0.393217,0.757907,0.780086,0.755441,0.572082,average,single
Datasets\abalone9-18.csv,0.366332,0.601129,0.699556,0.648083,0.564886,average,single
...,...,...,...,...,...,...,...
Datasets\yeast-2_vs_8.csv,0.847224,0.631738,0.904886,0.90916,0.505533,centroid,ward
Datasets\yeast1.csv,0.816898,0.70078,0.884848,0.883146,0.502394,average,ward
Datasets\yeast3.csv,0.816898,0.70078,0.884848,0.883146,0.502394,average,ward
Datasets\yeast4.csv,0.816898,0.70078,0.884848,0.883146,0.502394,average,ward


In [5]:
df.to_csv('CPCC.csv',)